# TCGA Breast Methylation Data Importation
**Local Version**: 1
**Source Version**: NA

This notebook will import raw TCGA methylation data through the [CGDS](http://www.cbioportal.org/cgds_r.jsp) portal for the study named "Breast Invasive Carcinoma (TCGA, Cell 2015)".

This study is preferred over "Breast Invasive Carcinoma (TCGA, Nature 2012)" despite the fact that it has a little fewer samples because it appears newer and includes more data types.

In [1]:
%run -m ipy_startup
%run -m ipy_logging
%matplotlib inline
from mgds.data_aggregation import database as db
from mgds.data_aggregation import source as src
from mgds.data_aggregation import api
from mgds.data_aggregation.import_lib import cgds
from mgds.data_aggregation.import_lib import tcga_breast
from py_utils.collection_utils import subset

In [2]:
case_list_id = tcga_breast.CASE_LIST_ID
genetic_profile_id = tcga_breast.PROF_METHYLATION
batch_size = 50

op = lambda: cgds.get_genetic_profile_data(
    case_list_id, genetic_profile_id,
    api.get_hugo_gene_ids(), gene_id_batch_size=batch_size
)
d = db.cache_raw_operation(op, src.TCGA_BREAST_v1, 'gene-methylation')

2016-11-24 08:29:14,242:DEBUG:mgds.data_aggregation.io_utils: Restoring serialized object from "/Users/eczech/data/research/mgds/raw/tcga-breast_v1_gene-methylation.pkl"


In [3]:
d.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39032 entries, 0 to 7
Columns: 819 entries, GENE_ID to TCGA-BH-A1ES-06
dtypes: float64(817), int64(1), object(1)
memory usage: 244.2+ MB


In [4]:
d = cgds.melt_raw_data(d)
d.info()

[Remove null values for column "VALUE"] Records before = 31889144, Records after = 8808287, Records removed = 23080857 (%72.38)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 8808287 entries, 0 to 31889143
Data columns (total 4 columns):
GENE_ID:CGDS    int64
GENE_ID:HGNC    object
CELL_LINE_ID    object
VALUE           float64
dtypes: float64(1), int64(1), object(2)
memory usage: 336.0+ MB


In [5]:
d_agg, d_dist = cgds.aggregate(d)
d_agg.head()

,CELL_LINE_ID,GENE_ID:HGNC,GENE_ID:CGDS,VALUE_MEAN,VALUE_STD,VALUE_CT
0,TCGA-A1-A0SB-01,A1BG,1,0.073395,0.0,1
1,TCGA-A1-A0SB-01,A1BG-AS1,503538,0.724501,0.0,1
2,TCGA-A1-A0SB-01,A2M,2,0.891226,0.0,1
3,TCGA-A1-A0SB-01,A2ML1,144568,0.478400,0.0,1
4,TCGA-A1-A0SB-01,A4GALT,53947,0.513391,0.0,1


In [6]:
d_dist

1    8785063
2      11612
Name: Number of Replicates, dtype: int64

In [7]:
d_agg.describe()

,GENE_ID:CGDS,VALUE_MEAN,VALUE_STD,VALUE_CT
count,8.796675e+06,8.796675e+06,8796675.0,8.796675e+06
mean,5.930480e+05,4.078580e-01,0.0,1.001320e+00
std,7.169097e+06,3.489149e-01,0.0,3.630843e-02
min,1.000000e+00,3.360104e-03,0.0,1.000000e+00
25%,6.774000e+03,4.721479e-02,0.0,1.000000e+00
50%,2.734700e+04,3.495343e-01,0.0,1.000000e+00
75%,8.466200e+04,7.720465e-01,0.0,1.000000e+00
max,1.005291e+08,9.962133e-01,0.0,2.000000e+00


In [8]:
assert np.all(pd.notnull(d_agg))
db.save(d_agg, src.TCGA_BREAST_v1, db.IMPORT, 'gene-methylation')

'/Users/eczech/data/research/mgds/import/tcga-breast_v1_gene-methylation.pkl'